In [1]:
import pandas as pd
import numpy as np
import sqlite3
from datetime import datetime
import logging
logging.basicConfig(level=logging.INFO)
DATABASE_DIR = "/Users/philips/Projects/deep-trading/crypto_db_to20180211_time30.db"

In [2]:
def select_coins(coin, start="2015/07/01", end="2018/02/11"):
    start_date = datetime.strptime(start,"%Y/%m/%d").timestamp()
    end_date = datetime.strptime(end,"%Y/%m/%d").timestamp()
    logging.info("select coins %s from %s to %s" % (coin, start, end))
    connection = sqlite3.connect(DATABASE_DIR)
    try:
        cursor=connection.cursor()
        cursor.execute('SELECT coin, date, open, high, low, close, volume FROM History WHERE'
                       ' date>=? and date<=? and coin=?'
                       ' ORDER BY coin, date ASC;',  #  LIMIT ?
                       (int(start_date), int(end_date), coin))
        coins_tuples = cursor.fetchall()
    finally:
        connection.commit()
        connection.close()
    coins = []
    for tuple in coins_tuples:
        coins.append(tuple[0])
    logging.debug("Selected coins are: "+str(coins))
    return coins, coins_tuples

In [3]:
def process_coin_df(coins_data):
    coins_data["Date_Time2"] = pd.to_datetime(coins_data["Date_Time"], unit="s")
    coins_data["Date"] = coins_data["Date_Time2"].dt.date
    coins_data["Time"] = coins_data["Date_Time2"].dt.time
    coins_data = coins_data.drop(["Date_Time", "Date_Time2"], axis=1)
    coins_data = coins_data.reindex(columns=["Currency", "Date", "Time", "Open", "High", "Low", "Close", "Volume"])
    return coins_data

def read_coin(coin, start="2016/07/01"):
    coins, coins_tuples = select_coins(start=start, coin=coin)
    coins_data = pd.DataFrame(coins_tuples, columns=["Currency", "Date_Time", "Open", "High", "Low", "Close", "Volume"])
    coins_data = process_coin_df(coins_data)
    return coins_data

In [15]:
coins_data = read_coin("ETH")
coins_data.to_csv("./data/BTCETH5.csv", index=False)
coins_data.head()

INFO:root:select coins ETH from 2016/07/01 to 2018/02/11


,Currency,Date,Time,Open,High,Low,Close,Volume
0,ETH,2016-06-30,21:00:00,0.018604,0.018650,0.018602,0.018603,8.518936
1,ETH,2016-06-30,21:05:00,0.018607,0.018649,0.018597,0.018598,42.692159
2,ETH,2016-06-30,21:10:00,0.018597,0.018600,0.018597,0.018599,2.027771
3,ETH,2016-06-30,21:15:00,0.018598,0.018650,0.018581,0.018595,20.319138
4,ETH,2016-06-30,21:20:00,0.018610,0.018626,0.018585,0.018601,5.432236


In [13]:
coins_data = read_coin("ZEC", start="2016/11/01")
coins_data.to_csv("./data/BTCZEC5.csv", index=False)
coins_data.tail()

INFO:root:select coins ZEC from 2016/11/01 to 2018/02/11


,Currency,Date_Time,Date_Time2,Date,Time,Open,High,Low,Close,Volume
134492,ZEC,1518298800,2018-02-10 21:40:00,2018-02-10,21:40:00,0.055901,0.055994,0.055755,0.055755,1.395733
134493,ZEC,1518299100,2018-02-10 21:45:00,2018-02-10,21:45:00,0.055929,0.056000,0.055805,0.056000,2.715299
134494,ZEC,1518299400,2018-02-10 21:50:00,2018-02-10,21:50:00,0.056110,0.056110,0.055789,0.055929,2.091979
134495,ZEC,1518299700,2018-02-10 21:55:00,2018-02-10,21:55:00,0.055929,0.056058,0.055805,0.055968,1.448158
134496,ZEC,1518300000,2018-02-10 22:00:00,2018-02-10,22:00:00,0.055825,0.056236,0.055682,0.056236,3.133552


In [6]:
coins_data = read_coin("reversed_USDT")
coins_data.to_csv("./data/BTCUSD5.csv", index=False)
coins_data.head()

INFO:root:select coins reversed_USDT from 2016/07/01 to 2018/02/11


,Currency,Date,Time,Open,High,Low,Close,Volume
0,reversed_USDT,2016-06-30,21:00:00,0.001497,0.001502,0.001491,0.001497,5.111943
1,reversed_USDT,2016-06-30,21:05:00,0.001497,0.001497,0.001497,0.001497,1.600599
2,reversed_USDT,2016-06-30,21:10:00,0.001500,0.001504,0.001499,0.001499,0.878455
3,reversed_USDT,2016-06-30,21:15:00,0.001497,0.001497,0.001492,0.001492,1.845085
4,reversed_USDT,2016-06-30,21:20:00,0.001492,0.001492,0.001492,0.001492,0.000000


In [7]:
coins_data = read_coin("reversed_USDT")
coins_data["Currency"] = "BTC"
coins_data["Open"] = 1/coins_data["Open"]
coins_data["High"] = 1/coins_data["High"]
coins_data["Low"] = 1/coins_data["Low"]
coins_data["Close"] = 1/coins_data["Close"]
coins_data.to_csv("./data/USDBTC5.csv", index=False)
coins_data.head()

INFO:root:select coins reversed_USDT from 2016/07/01 to 2018/02/11


,Currency,Date,Time,Open,High,Low,Close,Volume
0,BTC,2016-06-30,21:00:00,668.000000,666.000000,670.500000,668.000000,5.111943
1,BTC,2016-06-30,21:05:00,668.000000,668.000000,668.000000,668.000000,1.600599
2,BTC,2016-06-30,21:10:00,666.500000,665.001000,667.000000,667.000000,0.878455
3,BTC,2016-06-30,21:15:00,668.000000,668.000000,670.388889,670.388889,1.845085
4,BTC,2016-06-30,21:20:00,670.388889,670.388889,670.388889,670.388889,0.000000


## Select coins for timeframe:

In [31]:
def get_sql(feature, start, end, period, coin):
    if feature == "Close":
        sql = ("SELECT date+300-{period} AS date_norm, close FROM History WHERE"
               " date_norm>={start} and date_norm<={end}"
               " and date_norm%{period}=0 and coin=\"{coin}\"".format(
            start=start, end=end, period=period, coin=coin))
    elif feature == "Open":
        sql = ("SELECT date AS date_norm, open FROM History WHERE"
               " date_norm>={start} and date_norm<={end}"
               " and date_norm%{period}=0 and coin=\"{coin}\"".format(
            start=start, end=end, period=period, coin=coin))
    elif feature == "Volume":
        sql = ("SELECT date_norm, SUM(volume)" +
               " FROM (SELECT date-(date%{period}) "
               "AS date_norm, volume, coin FROM History)"
               " WHERE date_norm>={start} and date_norm<={end} and coin=\"{coin}\""
               " GROUP BY date_norm".format(
                   period=period, start=start, end=end, coin=coin))
    elif feature == "High":
        sql = ("SELECT date_norm, MAX(high)" +
               " FROM (SELECT date-(date%{period})"
               " AS date_norm, high, coin FROM History)"
               " WHERE date_norm>={start} and date_norm<={end} and coin=\"{coin}\""
               " GROUP BY date_norm".format(
                   period=period, start=start, end=end, coin=coin))
    elif feature == "Low":
        sql = ("SELECT date_norm, MIN(low)" +
               " FROM (SELECT date-(date%{period})"
               " AS date_norm, low, coin FROM History)"
               " WHERE date_norm>={start} and date_norm<={end} and coin=\"{coin}\""
               " GROUP BY date_norm".format(
                   period=period, start=start, end=end, coin=coin))
    else:
        msg = ("The feature %s is not supported" % feature)
        logging.error(msg)
        raise ValueError(msg)
    return sql


def select_coins_period(coin, period, features, start_date="2015/07/01", end_date="2018/02/11"):
    start = datetime.strptime(start_date,"%Y/%m/%d").timestamp()
    end = datetime.strptime(end_date,"%Y/%m/%d").timestamp()
    start = int(start - (start % period))
    end = int(end - (end % period))
    
    print("Collecting coin", coin, "from DB")
    connection = sqlite3.connect(DATABASE_DIR)
    all_tuples = []
    try:
        cursor=connection.cursor()
        for feature in features:
            sql = get_sql(feature, start, end, period, coin)
            cursor.execute(sql)
            coins_tuples = cursor.fetchall()
            all_tuples.append(coins_tuples)
    finally:
        connection.commit()
        connection.close()
    #from IPython.core.debugger import Tracer; Tracer()()
    length = len(all_tuples[0])-1
    date_column = np.array(all_tuples[0])[:length,0]
    print(date_column.shape)
    values_columns = np.array([np.array(all_tuples[i])[:length,1] for i in range(5)])
    print(values_columns.shape)
    return np.insert(values_columns, 0, date_column, axis=0)
        
def read_coin_period(coin, start="2016/07/01", period_mins=30):
    features = ["Open", "High", "Low", "Close", "Volume"]
    coins_array = select_coins_period(start_date=start, coin=coin, period=period_mins * 60, features=features)
    columns = ["Date_Time"] + features
    coins_data = pd.DataFrame(coins_array.T, columns=columns)
    coins_data["Currency"] = coin
    coins_data = process_coin_df(coins_data)
    return coins_data

In [43]:
coin = "ZEC"
period_mins = 5
coins_data = read_coin_period(coin, start="2016/11/01", period_mins=period_mins)
coins_data.to_csv("./data/BTC{}{}.csv".format(coin, period_mins), index=False)
coins_data[:6]

(134496,)
(5, 134496)


,Currency,Date,Time,Open,High,Low,Close,Volume
0,ZEC,2016-10-31,22:00:00,2.999999,3.400000,2.965000,3.20,149.828157
1,ZEC,2016-10-31,22:05:00,3.237312,3.300000,3.004893,3.25,76.784953
2,ZEC,2016-10-31,22:10:00,3.250000,3.290000,3.000000,3.08,76.251006
3,ZEC,2016-10-31,22:15:00,3.080000,3.100000,2.898796,2.90,83.367652
4,ZEC,2016-10-31,22:20:00,2.980000,2.999991,2.800000,2.97,57.346976
5,ZEC,2016-10-31,22:25:00,2.980000,3.160000,2.871809,3.09,101.725606


In [49]:
coin = "ZEC"
period_mins = 60
coins_data = read_coin_period(coin, start="2016/11/01", period_mins=period_mins)
coins_data.to_csv("./data/BTC{}{}.csv".format(coin, period_mins), index=False)
coins_data.head()

(11208,)
(5, 11208)


,Currency,Date,Time,Open,High,Low,Close,Volume
0,ZEC,2016-10-31,22:00:00,2.999999,3.4000,2.370000,2.575100,925.733908
1,ZEC,2016-10-31,23:00:00,2.575100,2.6000,2.065848,2.300000,504.751380
2,ZEC,2016-11-01,00:00:00,2.300000,2.3200,1.785000,1.807208,206.042436
3,ZEC,2016-11-01,01:00:00,1.807208,2.2500,1.685000,1.860000,178.513938
4,ZEC,2016-11-01,02:00:00,1.866528,1.9322,1.597985,1.710000,134.890048


In [50]:
coin = "ETH"
period_mins = 60
coins_data = read_coin_period(coin, period_mins=period_mins)
coins_data.to_csv("./data/BTC{}{}.csv".format(coin, period_mins), index=False)
coins_data.head()

(14161,)
(5, 14161)


,Currency,Date,Time,Open,High,Low,Close,Volume
0,ETH,2016-06-30,21:00:00,0.018604,0.018650,0.018572,0.018600,128.203343
1,ETH,2016-06-30,22:00:00,0.018600,0.018995,0.018504,0.018581,534.143036
2,ETH,2016-06-30,23:00:00,0.018581,0.018615,0.018450,0.018500,207.458189
3,ETH,2016-07-01,00:00:00,0.018500,0.018620,0.018436,0.018551,301.937187
4,ETH,2016-07-01,01:00:00,0.018551,0.018600,0.018430,0.018430,215.630485


In [51]:
coin = "reversed_USDT"
period_mins = 60
coins_data = read_coin_period(coin, period_mins=period_mins)
coins_data.to_csv("./data/BTCUSD{}.csv".format(period_mins), index=False)
coins_data.head()

(14161,)
(5, 14161)


,Currency,Date,Time,Open,High,Low,Close,Volume
0,reversed_USDT,2016-06-30,21:00:00,0.001497,0.001504,0.001491,0.001493,20.005475
1,reversed_USDT,2016-06-30,22:00:00,0.001493,0.001499,0.001491,0.001492,48.570146
2,reversed_USDT,2016-06-30,23:00:00,0.001492,0.001497,0.001480,0.001480,56.006210
3,reversed_USDT,2016-07-01,00:00:00,0.001480,0.001499,0.001480,0.001493,33.668783
4,reversed_USDT,2016-07-01,01:00:00,0.001496,0.001497,0.001481,0.001481,48.024391


In [52]:
coin = "reversed_USDT"
period_mins = 60
coins_data = read_coin_period(coin, period_mins=period_mins)
coins_data["Currency"] = "BTC"
coins_data["Open"] = 1/coins_data["Open"]
coins_data["High"] = 1/coins_data["High"]
coins_data["Low"] = 1/coins_data["Low"]
coins_data["Close"] = 1/coins_data["Close"]
coins_data.to_csv("./data/USDBTC{}.csv".format(period_mins), index=False)
coins_data.head()

(14161,)
(5, 14161)


,Currency,Date,Time,Open,High,Low,Close,Volume
0,BTC,2016-06-30,21:00:00,668.000,665.0010,670.500000,670.00,20.005475
1,BTC,2016-06-30,22:00:00,670.000,667.1000,670.800000,670.35,48.570146
2,BTC,2016-06-30,23:00:00,670.350,668.0000,675.505216,675.50,56.006210
3,BTC,2016-07-01,00:00:00,675.500,667.0000,675.500000,670.00,33.668783
4,BTC,2016-07-01,01:00:00,668.528,667.9516,675.200000,675.20,48.024391
